In [48]:
import os, sys, h5py

import numpy as np
import pandas as pd

import requests as rq
import io, h5py

import tensorflow as tf

In [49]:
link = 'http://jaspar.genereg.net/download/CORE/JASPAR2020_CORE_vertebrates_non-redundant_pfms_jaspar.txt' # jaspar file for CORE vertebrates (non redundant)
data = rq.get(link)
data.raise_for_status()

file = io.BytesIO(data.content)
jaspar_motifs = file.read().decode('UTF-8)') # raw jaspar txt file of all the motifs

In [50]:
split_file = jaspar_motifs.split('\n')[:-1]
motifs = [split_file[i*5+1:i*5+5] for i in range(int(len(split_file) / 5))]
motif_names = np.array([split_file[i*5].split('\t')[1].lower() for i in range(int(len(split_file) / 5))])

for i in range(len(motifs)):
    for j in range(len(motifs[i])):
        int_values = []
        for k in motifs[i][j].split(' '):
            try:
                int_values.append(int(k))
            except:
                continue
        motifs[i][j] = np.array(int_values)

pfms = np.array(motifs) # motif nucleotide position
ppms = []

for i in range(pfms.shape[0]):
    pfm = np.array(list(pfms[i])).transpose()
    ppm = np.array([pfm[j]/np.sum(pfm[j]) for j in range(pfm.shape[0])])
    ppms.append(ppm)
motif_ppms = np.array(ppms)

# motif_names
# motif_ppms

In [51]:
names = ['CEBPB', 'FOSL1', 'GABPA', 'GATA1', 'MEF2A', 'MAX', 'NFYB', 'SP1', 'SRF', 'STAT1', 'GATA1::TAL1', 'YY1']
# substituted Arid3a and MAFK for GATA1 and GATA1::TAL1

strand_motifs = []
core_index = []
for name in names:
    index = list(motif_names).index(name.lower())
    core_index.append(index)
    strand_motifs.append(motif_ppms[index])

# generate reverse compliments
core_motifs = []
core_names = []
for i in range(len(strand_motifs)):
    core_motifs.append(strand_motifs[i])
    core_names.append(names[i])
    reverse = strand_motifs[i][:,::-1]
    core_motifs.append(reverse[::-1,:])
    core_names.append(names[i] + '-rc')

In [52]:
def generate_random_sequence(length):
    sequence = []
    for i in range(length):
        sequence.append([0 for i in range(4)])
        sequence[i][int(np.random.rand()*4)] = 1
    return np.array(sequence)

In [53]:
def generate_random_binding_site(ppm):
    tf_seq = []
    for i in range(len(ppm)):
        nucleotide = np.random.choice([0, 1, 2, 3], 1, p=ppm[i])[0]
        tf_seq.append([0 for i in range(4)])
        tf_seq[i][nucleotide] = 1
    return np.array(tf_seq)

In [54]:
def embed_binding_site(sequence, binding_site, loc=None):
    if loc is None:
        loc = int(np.random.rand() * (len(sequence) - len(binding_site)))
    sequence[loc:loc+len(binding_site)] = binding_site
    return sequence

In [55]:
def create_convolution(seq_length, kernel_size, ppms):
    filters = len(ppms)
    
    inputs = tf.keras.Input(shape=(seq_length, 4))
    conv = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, use_bias=False)
    model = tf.keras.Model(inputs=inputs, outputs=conv(inputs))
    
    # initialize filters to look for target motifs
    filter_weights = []
    for i in range(len(ppms)):
        weights = np.vstack((ppms[i] - 0.25, np.zeros((kernel_size-len(ppms[i]), 4))))
        filter_weights.append(weights)
    filter_weights = np.array(filter_weights).transpose([1, 2, 0])
    
    conv.set_weights([filter_weights])
    
    return model

In [56]:
def get_false_positives(model, sequence, ppms, true_pos, thresh):
    out = model.predict(np.array([sequence])).transpose([2, 1, 0]).reshape((len(ppms), len(sequence)-model.layers[1].kernel_size[0]+1))
    
    all_positives = []
    for i in range(len(out)):
        significant = thresh * len(ppms[i])
        positives = np.where(out[i] > significant)[0]
        positives = [j for j in positives if j != true_pos[i]]
        all_positives.append(np.array(positives))
    all_positives = np.array(all_positives)
    
    return all_positives

In [57]:
def remove_false_positives(sequence, false_positives, true_pos, ppms):
    
    # iterate through false positives and replace with random sequences
    for i in range(len(false_positives)):
        for j in range(len(false_positives[i])):
            replace_start = false_positives[i][j]
            replace_end = false_positives[i][j] + len(ppms[i])

            # make sure the sequence does not overlap with a true positive
            change_bs = -1
            for k in range(len(true_pos)):
                if false_positives[i][j] < true_pos[k] and false_positives[i][j] + len(ppms[k]) - 1 > true_pos[k]:
                    replace_end = true_pos[k] - 1
                    if replace_end - replace_start <= len(ppms[k])/3:
                        change_bs = true_pos[k]
                elif false_positives[i][j] > true_pos[k] and false_positives[i][j] < true_pos[k] + len(ppms[k]) - 1:
                    replace_start = true_pos[k] + len(ppms[k])
                    if replace_end - replace_start <= len(ppms)/3:
                        change_bs = true_pos[k]
            if change_bs != -1:
                sequence[true_pos[k]:true_pos[k]+len(ppms[k])] = generate_random_binding_site(ppms[k])
            else:
                sequence[replace_start:replace_end] = generate_random_sequence(replace_end - replace_start)
    return sequence

In [58]:
def get_positions(seq_length, ppms, mu, sigma):
    # randomize space between interacting motifs
    spaces = []
    for i in range(len(ppms)):
        spaces.append([])
        for j in range(len(ppms[i]) - 1):
            spaces[i].append(int(np.random.normal(mu, sigma)))
            
    # compute total space required for all motifs (+1 padding)
    total_length = 1
    for i in range(len(ppms)):
        total_length += 1
        for j in range(len(ppms[i])):
            total_length += len(ppms[i][j])
            if j > 0:
                total_length += spaces[i][j-1]
    if total_length > seq_length:
        return None
    
    ppms = np.array(ppms)
    spaces = np.array(spaces)
    
    # shuffle motifs
    permutation = np.random.permutation(len(ppms))
    original_ppms = ppms
    ppms = ppms[permutation]
    spaces = spaces[permutation]
    
    # initialize location variables
    locs = []
    next_loc = 0
    bank = seq_length - total_length
    
    # randomize positions for all motifs
    rand = int(np.random.rand() * bank)
    next_loc += rand
    bank -= rand
    for i in range(len(ppms)):
        locs.append([])
        next_loc += 1
        
        rand = int(np.random.rand() * bank)
        next_loc += rand
        bank -= rand
        for j in range(len(ppms[i])):
            locs[i].append(next_loc)
            
            next_loc += len(ppms[i][j])
            if j < len(spaces[i]):
                next_loc += spaces[i][j]
    
    # rearrange locations to originl motifs order
    _locs = ppms
    _locs[permutation] = locs
    
    return _locs

In [62]:
def generate_embedded_sequence(seq_length, motifs, mu, sigma, thresh):
    kernel_size = 19
    
    # construct background sequence
    sequence = generate_random_sequence(seq_length)
    
    # randomize motif locations
    positions = get_positions(seq_length, motifs, mu, sigma)
    
    # construct and embed binding sites using the motif ppms
    motifs_flat = [] # used later for convolution
    true_pos_flat = []
    for i in range(len(motifs)):
        for j in range(len(motifs[i])):
            binding_site = generate_random_binding_site(motifs[i][j])
            sequence = embed_binding_site(sequence, binding_site, positions[i][j])
            
            motifs_flat.append(motifs[i][j])
            true_pos_flat.append(positions[i][j])
    
    conv = create_convolution(seq_length, kernel_size, motifs_flat)
    fps = get_false_positives(conv, sequence, motifs_flat, true_pos_flat, thresh)
    
    count = 0
    while len(fps.flatten()) > 0:
        sequence = remove_false_positives(sequence, fps, true_pos_flat, motifs_flat)
        fps = get_false_positives(conv, sequence, motifs_flat, true_pos_flat, thresh)
        count += 1
        if count > 10:
            print('generating diff sequence')
            return generate_embedded_sequence(seq_length, motifs, mu, sigma, thresh)

    return sequence

In [63]:
motifs = [[core_motifs[0]], [core_motifs[1], core_motifs[2]], [core_motifs[3], core_motifs[4], core_motifs[5]]]

generate_embedded_sequence(200, motifs, 15, 2, thresh=0.3) # experiment with different thresholds

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0,